In [30]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
# 최신 LangChain 기준
from langchain_core.documents import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import gradio as gr
from PyPDF2 import PdfReader
from docx import Document
# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [48]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv("SERP_API_KEY")
perplexity_api_key = os.getenv("PEPLEXITY_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"serp_api_key exists and begins {serp_api_key[:8]}")
else:
    print("serp_api_key not set")

if perplexity_api_key:
    print(f"perplexity_api_key exists and begins {perplexity_api_key[:8]}")
else:
    print("perplexity_api_key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBU
serp_api_key exists and begins c3ee9cec
perplexity_api_key exists and begins pplx-r4f


In [32]:
openai = OpenAI()
MODEL = 'gpt-4o'

In [33]:
def extract_text_from_file(file_path):
    """
    PDF 또는 Word(.docx) 파일에서 텍스트를 추출하는 함수

    Args:
        file_path (str): 파일 경로 (PDF 또는 DOCX)

    Returns:
        str: 추출된 텍스트
    """
    ext = os.path.splitext(file_path)[-1].lower()
    text = ""

    if ext == ".pdf":
        try:
            reader = PdfReader(file_path)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        except Exception as e:
            print(f"[PDF 읽기 오류] {e}")

    elif ext == ".docx":
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text += para.text + "\n"
        except Exception as e:
            print(f"[DOCX 읽기 오류] {e}")

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 가능합니다.")

    return text.strip()


In [34]:
# PDF 파일 경로
pdf_path = "RFP_ex.pdf"
rfp_text = extract_text_from_file(pdf_path)
rfp_text[:100]

'※ 본 제안요청서는  메리츠종금증권  웹접근성  개선 프로젝트를  위한 제안요청  목적으로  제공하는  \n것으로 , 그 외의 목적으로  사용할 수 없으며  외부 또는 타사에 배포할'

In [35]:
def get_user_input(
    rfp_text=None,
    style_selected=None,
    keywords_input=None,
    client_name=None,
    proposal_title=None,
    user_direction=None
):
    """
    사용자 입력 기반 제안서 생성용 입력값 정리 함수

    Args:
        rfp_text (str): RFP 원문 텍스트
        style_selected (str): 제안서 스타일 ("격식 있는", "신뢰감 있는" 등)
        keywords_input (str): 강조 키워드 쉼표 구분 (예: "AI, LLM, 효율성")
        client_name (str): 고객사명
        proposal_title (str): 제안서 제목
        user_direction (str): 고객 요청 방향성

    Returns:
        dict: 제안서 생성용 파라미터
    """

    if not rfp_text:
        raise ValueError("⚠️ RFP 텍스트는 필수입니다.")

    style_selected = style_selected or "신뢰감 있는"
    keywords_list = [kw.strip() for kw in (keywords_input or "").split(",") if kw.strip()]
    client_name = client_name or "고객사명 미입력"
    proposal_title = proposal_title or "제안서 제목 미입력"
    user_direction = user_direction or ""

    return {
        "rfp_text": rfp_text,
        "style": style_selected,
        "keywords": keywords_list,
        "client_name": client_name,
        "proposal_title": proposal_title,
        "user_direction": user_direction
    }


In [36]:
# ✅ 예시로 RFP 텍스트와 일부 값들을 입력해 실험
sample_rfp_text = rfp_text

user_inputs = get_user_input(
    rfp_text=sample_rfp_text,
    style_selected="근거가 있고 formal하게",
    keywords_input="타사 비교, 자체적인, 벤치마크",
    client_name="EY 컨설팅",
    proposal_title="메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서",
    user_direction="현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요"
)

# ✅ 출력 확인
for k, v in user_inputs.items():
    print(f"\n🔹 {k}:\n{v if not isinstance(v, str) else v[:50]}")



🔹 rfp_text:
※ 본 제안요청서는  메리츠종금증권  웹접근성  개선 프로젝트를  위한 제안요청  목적으로

🔹 style:
근거가 있고 formal하게

🔹 keywords:
['타사 비교', '자체적인', '벤치마크']

🔹 client_name:
EY 컨설팅

🔹 proposal_title:
메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서

🔹 user_direction:
현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요


In [102]:
PROPOSAL_SLIDE_TEMPLATES = {
    "cover_page": {
        "slide_description": "프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명, 고객사명, 작성자 및 로고를 명확히 배치하여 첫 인상을 결정짓습니다.",
        "description": "프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다.",
        "elements": {
            "Title": "프로젝트의 정식 명칭 (중앙 상단에 크게 배치)",
            "Subtitle": "고객사명 또는 부제 설명 (Title 아래 위치)",
            "ProjectDate": "제안서 작성 또는 제출 일자 (하단 우측 또는 좌측 구석에 배치)",
            "PreparedBy": "작성자 또는 제안 주체 (ProjectDate 인근 또는 하단 중앙)",
            "Logo": "회사 또는 고객사 로고 (우상단 또는 좌상단에 적절히 배치)"
        }
    },
    "executive_summary": {
        "slide_description": "제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.",
        "description": "전체 제안서의 핵심 내용을 1~2페이지에 요약하여 고위 임원에게 전달합니다.",
        "elements": {
            "Title": "슬라이드 제목 (예: 제안 개요, Executive Summary)",
            "SummaryPoints": "핵심 제안 내용 Bullet (전략, 기대 효과, 기간, 투자 규모 등)",
            "ClientValue": "고객에게 제공되는 핵심 가치 또는 차별화된 이점 요약"
        }
    },
    "table_of_contents": {
        "slide_description": "전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.",
        "description": "전체 제안서의 구성 흐름을 한눈에 보여주는 목차입니다.",
        "elements": {
            "Title": "목차 제목 (상단 중앙)",
            "SectionList": "슬라이드별 주요 제목 리스트 (Bullet 형식으로 왼쪽 정렬)"
        }
    },
    "project_understanding": {
        "slide_description": "프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있다는 점을 전달합니다.",
        "description": "프로젝트의 배경 및 목적을 설명하고 고객의 니즈에 대한 이해를 보여줍니다.",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙)",
            "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스)",
            "KeyObjectives": "고객의 주요 목표 목록 (왼쪽 열에 Bullet 형식)",
            "BackgroundIssues": "해결하고자 하는 문제 또는 현재 상황 (오른쪽 열에 Bullet 형식)"
        }
    },
    "client_needs_summary": {
        "slide_description": "고객의 구체적인 요구사항을 명확하게 정리해 실무적 방향성과 대응의 기준을 제시합니다.",
        "description": "고객이 제시한 주요 요구사항을 요약하고 정리합니다.",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙)",
            "MiddleText": "요구사항 요약 핵심 문장",
            "BulletPoints": "고객 니즈 요약 문장 리스트",
            "NeedsMatrix": "요구사항을 정리한 표 (요구사항 | 중요도 | 현재 상태)"
        }
    },
    "market_analysis_market_overview": {
        "slide_description": "해당 산업의 크기, 구조, 주요 트렌드를 데이터와 함께 제시해 전체 시장 배경을 설명합니다.",
        "description": "해당 산업 및 시장의 전반적인 개요를 제공합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "시장 개요 요약 문장",
            "GraphLeft": "시장 관련 그래프 (예: 점유율, 성장률 등)",
            "TextRight": "시장 현황 및 주요 특징 설명 텍스트"
        }
    },
    "growth_trend_analysis": {
        "slide_description": "시장 혹은 기술의 성장세를 데이터 기반으로 시각화하고 그 시사점을 제시합니다.",
        "description": "시장 또는 기술의 성장 추세를 시각적으로 설명합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "성장 트렌드 핵심 요약 문장",
            "GraphLeft": "성장 추이 그래프 (선형, 누적 등)",
            "TextRight": "그래프 해석 및 성장 요인 서술"
        }
    },
    "industry_drivers_challenges": {
        "slide_description": "산업 내 성장 요인과 저해 요인을 대조해 전략 방향 설정의 기초로 삼습니다.",
        "description": "산업을 이끄는 주요 동인과 직면한 도전 과제를 명확히 분리해 비교합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "산업 변화 요약 또는 주요 시사점",
            "DriversList": "산업 동인 목록 (왼쪽 컬럼 Bullet 형식)",
            "ChallengesList": "산업 장애 요소 목록 (오른쪽 컬럼 Bullet 형식)",
            "DetailComments": "각 항목에 대한 보충 설명 또는 전략적 인사이트"
        }
    },
    "competitive_benchmarking": {
        "slide_description": "경쟁사 대비 자사의 상대적 위치와 우위를 정량적·정성적으로 설명합니다.",
        "description": "주요 경쟁사와의 비교를 통해 자사의 경쟁력을 강조합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "경쟁 포지셔닝 요약",
            "TableMain": "경쟁사 비교 요약 테이블",
            "BenchmarkTable": "세부 비교 테이블 (경쟁사 별 행, 항목 별 열 구성)",
            "CompetitiveInsights": "경쟁사 전략 및 고객 시사점 도출",
            "TextBottom": "요약 또는 인사이트 도출 문장"
        }
    },
    "swot_analysis": {
        "slide_description": "내부·외부 환경 분석을 통해 전략적 포지션을 진단하고 방향성을 제시합니다.",
        "description": "Strength, Weakness, Opportunity, Threat의 네 가지 측면에서 분석합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "SWOT 분석 요약 문장",
            "SWOT-S": "Strength 요소 리스트",
            "SWOT-W": "Weakness 요소 리스트",
            "SWOT-O": "Opportunity 요소 리스트",
            "SWOT-T": "Threat 요소 리스트",
            "NarrativeSummary": "요약 설명 또는 전략적 시사점"
        }
    },
    "solution_overview": {
        "slide_description": "제안 솔루션의 핵심 가치와 구조를 시각화하고 간명하게 전달합니다.",
        "description": "제안하는 솔루션의 전체 구조와 핵심 기능을 요약합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "솔루션 개요 및 핵심 가치",
            "SimpleDiagram": "솔루션 흐름도 또는 구조도",
            "ArchitectureDiagram": "기술 아키텍처 또는 시스템 구성도 (선택적)"
        }
    },
    "use_case_scenarios": {
        "slide_description": "솔루션이 실제로 어떻게 작동하고 활용될 수 있는지를 시나리오 기반으로 보여줍니다.",
        "description": "제안 솔루션이 실제로 어떻게 적용되는지 시나리오 형태로 설명합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "ScenarioDiagram": "워크플로우 또는 사용자 여정 흐름도",
            "NarrativeCases": "상황 기반 시나리오 설명"
        }
    },
    "strategic_recommendations": {
        "slide_description": "고객을 위한 전략적 방향성과 구체 실행안, 우선순위를 제시합니다.",
        "description": "제안사의 전략적 방향성과 구체적인 실행 방안을 제시합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "전략 방향 요약",
            "BulletPoints": "주요 전략 항목",
            "PriorityMap": "2x2 우선순위 매트릭스"
        }
    },
    "implementation_plan": {
        "slide_description": "전략 실행을 위한 단계별 로드맵을 구체적으로 설명합니다.",
        "description": "솔루션 실행을 위한 구체적인 단계와 계획을 제시합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "실행 로드맵 핵심 요약",
            "TimelineMain": "전체 실행 일정 요약",
            "Phases": "단계별 세부 계획",
            "TextBottom": "리스크 또는 보완 고려사항"
        }
    },
    "timeline_milestones": {
        "slide_description": "전체 일정에서의 주요 마일스톤을 시각화하여 이해도를 높입니다.",
        "description": "주요 일정과 마일스톤을 시각적으로 정리합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "일정 요약 또는 주요 포인트",
            "GanttChart": "간트 차트 또는 일정 도식"
        }
    },
    "risk_management_plan": {
        "slide_description": "예상되는 리스크를 식별하고 이에 대한 대응 전략을 제시합니다.",
        "description": "프로젝트 리스크와 그에 대한 대응 방안을 제시합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "리스크 관리 개요",
            "RiskTable": "리스크 목록 (내용 | 가능성 | 영향도)",
            "MitigationStrategy": "완화 전략 항목"
        }
    },
    "expected_benefits": {
        "slide_description": "제안서 실행 시 고객이 얻게 될 주요 기대 효과를 정성·정량적으로 표현합니다.",
        "description": "고객이 얻게 될 기대 효과와 정량적 지표를 설명합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "핵심 기대 효과 요약",
            "BulletPoints": "기대 효과 리스트",
            "KPIProjection": "성과 지표 추이 그래프 또는 테이블"
        }
    },
    "investment_budget_estimation": {
        "slide_description": "전체 예산과 각 항목별 비용을 상세히 제시하여 투자 가시성을 제공합니다.",
        "description": "프로젝트에 필요한 예산 및 비용 항목을 정리합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "투자 요약",
            "BudgetTable": "총 예산 테이블",
            "CostBreakdown": "비용 항목별 상세 내용"
        }
    },
    "team_introduction": {
        "slide_description": "수행팀의 전문성과 역할 분담을 보여줘 신뢰도를 높입니다.",
        "description": "참여 인력 및 팀의 역량을 소개합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "팀 구성 및 핵심 인력 요약",
            "MemberList": "핵심 인력 소개 (이름, 역할, 경력 등)",
            "TeamOrgChart": "팀 조직도"
        }
    },
    "why_us_differentiation": {
        "slide_description": "자사만의 차별성과 강점을 강조하여 경쟁사 대비 우위를 설득합니다.",
        "description": "자사의 강점과 차별성을 경쟁사와 비교하여 강조합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "자사 강점 요약",
            "BulletPoints": "차별화 요소 리스트",
            "ComparisonTable": "자사 vs 경쟁사 비교 표"
        }
    },
    "closing_summary": {
        "slide_description": "전체 제안을 요약하며 고객에게 남기고 싶은 핵심 메시지를 강조합니다.",
        "description": "제안 내용을 요약하고 핵심 메시지를 다시 강조합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "핵심 요약 문장",
            "FinalCall": "콜투액션 또는 향후 협업 제안"
        }
    },
    "qna": {
        "slide_description": "질의응답을 위한 충분한 공간과 예상 질문 대응 내용을 구성합니다.",
        "description": "질의응답을 위한 빈 공간과 예상 질문을 포함합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "EmptySpace": "Q&A 시각적 공간",
            "AnticipatedQuestions": "예상 질문 및 답변 리스트"
        }
    },
    "appendix": {
        "slide_description": "통계, 기술 스펙, 참고 문헌 등 본문에 넣기 어려운 자료를 정리합니다.",
        "description": "참고 자료, 기술 상세, 기타 부속 정보를 수록합니다.",
        "elements": {
            "Title": "슬라이드 제목",
            "SupportingDetails": "기술 자료, 통계 인용 등",
            "Footnotes": "출처, 링크, 각주 등 보충 정보"
        }
    }
}


In [103]:
def analyze_rfp(
    user_inputs: dict,
    slide_templates: dict,
    model="gpt-4o",
    temperature=0.3
):
    """
    get_user_input 결과 dict를 받아 GPT로 분석하여 PPT 슬라이드 구조 생성

    Args:
        user_inputs (dict): get_user_input 함수에서 반환된 사용자 입력값
        slide_templates (dict): PPT 템플릿 구조
        model (str): 사용할 OpenAI 모델
        temperature (float): 생성 온도

    Returns:
        dict: 슬라이드별 제안서 구성 내용
    """
    import openai
    import json
    import re

    rfp_text = user_inputs.get("rfp_text")
    style = user_inputs.get("style", "신뢰감 있는")
    keywords = user_inputs.get("keywords", [])
    client_name = user_inputs.get("client_name", "고객사명 미입력")
    proposal_title = user_inputs.get("proposal_title", "제안서 제목 미입력")
    user_direction = user_inputs.get("user_direction", "")

    if not rfp_text or len(rfp_text.strip()) < 30:
        raise ValueError("❗ RFP 원문이 비어 있거나 너무 짧습니다. 실제 RFP를 반드시 입력하세요.")
    if slide_templates is None:
        raise ValueError("PPT 템플릿 구조 딕셔너리가 필요합니다.")

    # system prompt
    system_prompt = f"""
당신은 'EY·맥킨지·삼성' 등 국내외 최상위 전략 컨설팅 회사의 파트너급 제안서 전문가 AI입니다.

아래 RFP 원문과 고객 요구를 바탕으로,
각 PPT 슬라이드별로 다음 기준에 따라 실제 컨설팅 현장에서 쓰일 수준의 상세하고 설득력 있는 제안서 초안 구조를 만드세요.
같은 템플릿이라도 내용이 많으면 반복 사용 가능합니다.

1. [description]
- 단순 목적 설명이 아닌: '왜 중요한가', '고객사 상황과의 연계성', '전략적 필요성', '차별화된 근거', '시장 및 경쟁사 변화와 연결성' 포함
- 시장 통계, 업계 사례, 정책·기술 트렌드, 고객사 현황 등 객관적 근거를 적극 인용
- 3~5문장 이상의 깊이 있는 발표용 컨설팅 워딩

2. [elements]
- 실제 발표자료처럼 사용 가능한 수준으로 표, 수치, 근거, 사례 등 포함
- 표는 다음 형식으로 작성:
  "표 제목": [["헤더1", "헤더2"], ["값1", "값2"]]
- 그래프는 다음 형식으로 작성:
  "그래프 제목": {{
    "description": "시장 성장률 추이 (2019~2024)",
    "graph_type": "Line Chart",
    "data_source": "Statista, 2023",
    "data_table": [["연도", "시장 규모 (억원)"], ["2019", "1200"], ["2020", "1400"]]
  }}

3. [needs_research]
- 외부 리서치가 필요한 항목(정책, 통계, 사례 등)을 질문 형식으로 구체화
- 단순 요약이 아니라 실질적 조사 질문이어야 함

⚡️ 모든 슬라이드는 전략적 논리, 정량/정성 근거, 차별화 요인, 사례와 수치, 실무 설득력 기반으로 작성되어야 하며, 평가자와 실무자가 모두 납득할 수 있어야 합니다.

[슬라이드 템플릿 구조]
{json.dumps(slide_templates, ensure_ascii=False, indent=2)}
""".strip()

    # user prompt
    user_prompt = f"""
[RFP 원문]
{rfp_text}

[고객 방향성/강조]
{user_direction or '없음'}

(고객명: {client_name} / 제안서 제목: {proposal_title})
[강조 키워드]: {', '.join(keywords) if keywords else '없음'}

위 정보를 바탕으로 PPT 슬라이드별 'description', 'elements', 'needs_research'를 작성해주세요.
""".strip()

    # GPT 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )

    result_text = response.choices[0].message.content.strip()

    # 코드블럭 제거
    result_text = re.sub(r"^```(?:json)?\s*|\s*```$", "", result_text, flags=re.DOTALL)

    try:
        parsed = json.loads(result_text)
        return parsed
    except json.JSONDecodeError as e:
        raise ValueError(f"응답이 유효한 JSON 형식이 아닙니다:\n\n{result_text}\n\n에러: {e}")


In [88]:
slides_dict = analyze_rfp(user_inputs=user_inputs, slide_templates=PROPOSAL_SLIDE_TEMPLATES)
slides_dict

{'cover_page': {'slide_description': '프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다.',
  'description': '메리츠종금증권 웹접근성 개선 프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다. 이는 프로젝트의 중요성과 고객사의 브랜드 이미지를 강조하는 첫인상입니다.',
  'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2014.02',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 및 메리츠종금증권 로고'}},
 'table_of_contents': {'slide_description': '전체 제안서의 구성 흐름을 한눈에 보여주는 목차입니다.',
  'description': '제안서의 전체 흐름을 한눈에 파악할 수 있도록 목차를 제공하여 독자가 쉽게 원하는 정보를 찾을 수 있도록 합니다. 이는 제안서의 체계성과 논리적 구성을 강조합니다.',
  'elements': {'Title': '목차',
   'SectionList': ['I. 사업개요',
    'II. 제안요청 내용',
    'III. 제안안내',
    'IV. 참여조건 및 방법',
    'V. 첨부']}},
 'project_understanding': {'slide_description': '프로젝트의 배경 및 목적을 설명하고 고객의 니즈에 대한 이해를 보여줍니다.',
  'description': '웹 접근성 개선의 필요성과 목표를 명확히 설명하여 프로젝트의 중요성과 시급성을 강조합니다. 특히 장차법 시행에 따른 법적 준수 필요성과 고객 사용 편의성 강화의 중요성을 부각합니다.',
  'elements': {'Title': '프로젝트 이해',
   'MiddleText': '웹 접근성 준수를 통한 법적 리스크 제거 및 사용자 편의성 강화

In [89]:
import requests

def search_perplexity(query, api_key):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": query}],
        "return_citations": True
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        try:
            data = response.json()
            message = data["choices"][0]["message"]
            content = message.get("content", "")
            citations = message.get("citations", [])
            urls = [c.get("url") for c in citations if "url" in c]
            return content or "", urls
        except Exception as e:
            print("Perplexity Parsing Error:", e)
            return "", []
    else:
        print("Perplexity API Error:", response.status_code, response.text)
        return "", []


def search_serpapi_with_url(query, serpapi_key):
    """
    SerpAPI를 사용하여 Google 검색 결과 요약과 함께 URL 반환
    """
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": serpapi_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()
        snippets = []
        urls = []
        for r in results.get("organic_results", [])[:3]:
            snippet = r.get("snippet", "")
            link = r.get("link", "")
            if snippet and link:
                snippets.append(snippet)
                urls.append(link)
        return "\n".join(snippets), urls
    else:
        print("SerpAPI Error:", response.status_code, response.text)
        return "", []

def check_relevance(description, research_result):
    """
    리서치 결과가 slide_description과 논리적으로 부합하는지 확인
    """
    prompt = (
        f"당신은 제안서 전략 검토 전문가입니다.\n"
        f"다음은 슬라이드의 목적(description)과 검색을 통해 얻은 리서치 결과입니다.\n"
        f"[슬라이드 목적]\n{description}\n\n"
        f"[리서치 결과]\n{research_result}\n\n"
        f"이 리서치 결과가 위 슬라이드 목적을 충분히 뒷받침합니까? "
        f"'네' 또는 '아니오'로 시작하고, 반드시 이유를 1문장 이상 포함하세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )
    answer = response.choices[0].message.content.strip()
    print(f"\n[GPT 판단 결과]\n{answer}\n")
    return answer.startswith("네")

def regenerate_query(original_query, previous_answer):
    """
    검색 결과가 부적합한 경우, 질문을 더 명확하게 만들어주는 로직
    """
    prompt = (
        f"다음은 정보 검색을 위한 원래 질문과 검색 결과입니다.\n\n"
        f"[질문]\n{original_query}\n\n"
        f"[검색 결과]\n{previous_answer}\n\n"
        "위 결과가 부정확하거나 부족한 경우, 질문을 더 구체적이고 답변을 유도할 수 있도록 재작성하세요. "
        "새 질문은 1문장으로 간결하게 작성해 주세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# def smart_research_fill(slides_dict, search_key, search_mode="perplexity", max_retry=2):
#     """
#     슬라이드에 대해 스마트 리서치를 수행하고, 불충분한 경우 재질문하여 더 나은 결과를 시도함
#     리서치 결과와 함께 URL도 저장
#     """
#     completed_results = {}

#     for slide_id, slide in slides_dict.items():
#         desc = slide.get("slide_description", "")
#         needs_list = slide.get("needs_research", [])
#         if not needs_list:
#             continue

#         slide_results = {}
#         for need in needs_list:
#             query = need
#             attempt = 0
#             while attempt < max_retry:
#                 if search_mode == "perplexity":
#                     result = search_perplexity(query, search_key)
#                     urls = []  # Perplexity는 URL 제공 안 함
#                 elif search_mode == "serpapi":
#                     result, urls = search_serpapi_with_url(query, search_key)
#                 else:
#                     raise ValueError("search_mode는 'perplexity' 또는 'serpapi'만 지원됩니다.")

#                 if not result:
#                     break

#                 if check_relevance(desc, result):
#                     slide_results[need] = {
#                         "content": result,
#                         "urls": urls
#                     }
#                     break  # 성공하면 종료
#                 else:
#                     query = regenerate_query(query, result)
#                     attempt += 1
#                     time.sleep(1)

#         if slide_results:
#             completed_results[slide_id] = {
#                 "slide_title": slide.get("slide_title", ""),
#                 "research_results": slide_results
#             }

#     return completed_results
def smart_research_fill(slides_dict, search_mode="perplexity", max_retry=2):
    """
    슬라이드에 대해 스마트 리서치를 수행하고, 불충분한 경우 재질문하여 더 나은 결과를 시도함.
    Perplexity 사용 시 citation URL도 함께 저장됨.
    """
    completed_results = {}

    for slide_id, slide in slides_dict.items():
        desc = slide.get("slide_description", "")
        if not desc:
            print(f"[경고] slide_description이 비어 있습니다: slide_id={slide_id}")
            continue

        needs_list = slide.get("needs_research", [])
        if not needs_list:
            continue

        slide_results = {}
        for need in needs_list:
            query = need
            attempt = 0

            while attempt < max_retry:
                # 검색 수행
                if search_mode == "perplexity":
                    content, urls = search_perplexity(query, perplexity_api_key)
                elif search_mode == "serpapi":
                    content, urls = search_serpapi_with_url(query, serp_api_key)
                else:
                    raise ValueError("search_mode는 'perplexity' 또는 'serpapi'만 지원됩니다.")

                # 실패 시 중단
                if not content:
                    print(f"[실패] 검색 결과 없음 - query: {query}")
                    break

                # GPT 판단으로 관련성 확인
                if check_relevance(desc, content):
                    slide_results[need] = {
                        "content": content.strip(),
                        "urls": urls
                    }
                    break  # 성공 → 다음 need로
                else:
                    query = regenerate_query(query, content)
                    attempt += 1
                    time.sleep(1)

        if slide_results:
            completed_results[slide_id] = {
                "slide_title": slide.get("slide_title", ""),
                "research_results": slide_results
            }

    return completed_results



def merge_research_results_into_slides(slides_dict, research_results_dict):
    """
    slides_dict 구조에 research_results_dict 데이터를 병합
    slides_dict[slide_id]['research_results'] 로 삽입
    """
    for slide_id, data in research_results_dict.items():
        if slide_id in slides_dict:
            slides_dict[slide_id]['research_results'] = data.get('research_results', {})
    return slides_dict


In [72]:
# # peplexity 테스트
# def search_perplexity(query, api_key):
#     url = "https://api.perplexity.ai/chat/completions"
#     headers = {
#         "Authorization": f"Bearer {api_key}",
#         "Content-Type": "application/json"
#     }
#     payload = {
#         "model": "sonar-pro",  # Use a valid model name
#         "messages": [{"role": "user", "content": query}]
#     }

#     response = requests.post(url, headers=headers, json=payload)
#     if response.status_code == 200:
#         data = response.json()
#         return data['choices'][0]['message']['content']
#     else:
#         print("Perplexity API Error:", response.status_code, response.text)
#         return ""

# test = search_perplexity("2024년 한국의 웹 접근성 관련 정책과 시장 동향 요약해줘", perplexity_api_key)
# print(test)


KeyboardInterrupt: 

In [90]:
import time
research_results_dict = smart_research_fill(slides_dict, search_mode="perplexity")

slides_dict = merge_research_results_into_slides(
    slides_dict=slides_dict,
    research_results_dict=research_results_dict
)

slides_dict


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 이유는 국내 웹 접근성 시장의 성장률, 도입 실태, 주요 사례, 정책 및 지원 동향 등을 포괄적으로 다루어 해당 산업 및 시장의 전반적인 개요를 제공하기 때문입니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드의 목적을 충분히 뒷받침합니다. ESG 규제와 법제화의 강화가 웹 접근성과 어떻게 연계되는지를 구체적으로 설명함으로써 산업 및 시장의 전반적인 개요를 제공하고 있습니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 웹 접근성 기술의 역사적 발전과 최신 트렌드를 시각적으로 설명함으로써 시장과 기술의 성장 추세를 명확하게 보여줄 수 있습니다.


[GPT 판단 결과]
아니오. 슬라이드의 목적은 시장 또는 기술의 성장 추세를 시각적으로 설명하는 것이지만, 제공된 리서치 결과는 주로 법적 요구사항 변화와 규제 강화에 대한 내용으로, 성장 추세를 직접적으로 설명하지 않습니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 법적 변화들이 디지털 전환 시대에 맞춰 금융 및 공공 부문에서의 성장 추세와 혁신을 촉진하는 방향으로 이루어지고 있음을 시각적으로 설명할 수 있습니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 주요 동인으로는 정보취약계층의 정보 접근권 보장을 위한 법적 규제 강화가 있으며, 도전 과제로는 이러한 규제를 준수하기 위한 기술적 및 운영적 변화가 필요하다는 점이 명확히 드러납니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 디지털 포용성과 관련된 주요 동인과 도전 과제를 명확히 구분하여 설명하고 있으며, 정부와 기업의 대응 전략을 통해 이러한 도전 과제를 어떻게 해결하고 있는지를 비교할 수 있게 해줍니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 

{'cover_page': {'slide_description': '프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다.',
  'description': '메리츠종금증권 웹접근성 개선 프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다. 이는 프로젝트의 중요성과 고객사의 브랜드 이미지를 강조하는 첫인상입니다.',
  'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2014.02',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 및 메리츠종금증권 로고'}},
 'table_of_contents': {'slide_description': '전체 제안서의 구성 흐름을 한눈에 보여주는 목차입니다.',
  'description': '제안서의 전체 흐름을 한눈에 파악할 수 있도록 목차를 제공하여 독자가 쉽게 원하는 정보를 찾을 수 있도록 합니다. 이는 제안서의 체계성과 논리적 구성을 강조합니다.',
  'elements': {'Title': '목차',
   'SectionList': ['I. 사업개요',
    'II. 제안요청 내용',
    'III. 제안안내',
    'IV. 참여조건 및 방법',
    'V. 첨부']}},
 'project_understanding': {'slide_description': '프로젝트의 배경 및 목적을 설명하고 고객의 니즈에 대한 이해를 보여줍니다.',
  'description': '웹 접근성 개선의 필요성과 목표를 명확히 설명하여 프로젝트의 중요성과 시급성을 강조합니다. 특히 장차법 시행에 따른 법적 준수 필요성과 고객 사용 편의성 강화의 중요성을 부각합니다.',
  'elements': {'Title': '프로젝트 이해',
   'MiddleText': '웹 접근성 준수를 통한 법적 리스크 제거 및 사용자 편의성 강화

In [23]:
# serpapi api 다떨어짐, 추후 perplexity 진행
slides_dict = {'client_needs_summary': {'elements': {'BulletPoints': ['웹 접근성 인증마크 획득', '모바일 및 다양한 브라우저 호환성 확보', '기존 시스템과의 원활한 연계'],
                                       'NeedsMatrix': [['요구사항', '중요도', '현재 상태'], ['웹 접근성 인증', '높음', '미흡'], ['모바일 호환성', '중간', '부분적'], ['시스템 연계', '높음', '미흡']],
                                       'Title': '고객 니즈 요약'},
                          'needs_research': ['경쟁사의 웹 접근성 인증 사례', '다양한 브라우저 및 모바일 기기 호환성 확보 방법'],
                          'research_results': {'경쟁사의 웹 접근성 인증 사례': {'content': "국내 최다 WA인증마크 획득, 스크린리더 연동(웹 접근성 준수 태그 사용), 크로스플랫폼, 멀티OS 지원 등 HTML5엔터프라이 UI플랫폼 '웹스퀘어5'.\n"
                                                                               '웹 접근성 준수 태그 사용으로. 경쟁사 대비 압도적인 인증/수행 사례 를 보유중에 있습니다. \u200b. ( 국세청 취업 후 학자금 상환 주택도시보증공사 ...\n'
                                                                               '뿐만 아니라 시각장애인도 스크린 리더(Screen Reader, 웹사이트 내용을 음성으로 안내하는 프로그램)를 연결하면 쉽게 음성서비스를 받을 수 있게 했다.',
                                                                    'urls': ['https://blog.naver.com/inswavemk/222985505220',
                                                                             'https://blog.naver.com/inswavemk/223486756708?viewType=pc',
                                                                             'https://news.lgdisplay.com/2013/05/lg%EB%94%94%EC%8A%A4%ED%94%8C%EB%A0%88%EC%9D%B4-%EC%97%85%EA%B3%84-%EC%B5%9C%EC%B4%88-%EC%9B%B9-%EC%A0%91%EA%B7%BC%EC%84%B1-%EC%9D%B8%EC%A6%9D%EB%A7%88%ED%81%AC-%ED%9A%8D%EB%93%9D/']}},
                          'slide_description': '고객의 요구사항을 명확히 파악하고, 이를 기반으로 한 맞춤형 솔루션을 제안합니다. 메리츠종금증권은 웹 접근성 개선을 통해 법적 준수를 넘어 고객 경험을 향상시키고자 합니다.',
                          'slide_title': '고객 니즈 요약'},
 'closing_summary': {'elements': {'FinalCall': 'EY 컨설팅과 함께 웹 접근성 개선을 통해 더 나은 고객 경험을 제공하세요.', 'MiddleText': '웹 접근성 개선을 통한 고객 경험 향상 및 법적 준수', 'Title': '제안 요약'},
                     'needs_research': ['웹 접근성 개선의 최종 기대 효과', 'EY 컨설팅의 차별화된 가치 제안'],
                     'slide_description': '제안서의 핵심 내용을 요약하여, 메리츠종금증권이 본 제안을 선택해야 하는 이유를 명확히 전달합니다.',
                     'slide_title': '제안 요약'},
 'competitive_benchmarking': {'elements': {'BenchmarkTable': {'Header': ['항목', '메리츠종금증권', '경쟁사 A', '경쟁사 B'],
                                                              'Rows': [['웹 접근성 인증', '미획득', '획득', '미획득'], ['모바일 호환성', '중간', '높음', '중간']]},
                                           'TableMain': [['경쟁사', '웹 접근성 인증', '모바일 호환성', '고객 만족도'], ['A사', '획득', '높음', '85'], ['B사', '미획득', '중간', '75']],
                                           'TextBottom': '경쟁사 A는 웹 접근성 인증을 통해 고객 만족도를 높였으며, 이는 메리츠종금증권이 참고할 수 있는 중요한 사례입니다.',
                                           'Title': '경쟁사 비교 분석'},
                              'needs_research': ['경쟁사의 웹 접근성 인증 및 개선 사례', '경쟁사와의 차별화 전략'],
                              'slide_description': '주요 경쟁사의 웹 접근성 개선 사례를 분석하여 메리츠종금증권이 벤치마킹할 수 있는 요소를 도출합니다.',
                              'slide_title': '경쟁사 비교 분석'},
 'cover_page': {'elements': {'Logo': '메리츠종금증권 로고', 'PreparedBy': 'EY 컨설팅', 'ProjectDate': '2014.02', 'Subtitle': '웹접근성 향상을 위한 전략적 제안', 'Title': '메리츠종금증권 웹접근성 개선 프로젝트 제안서'},
                'slide_title': '커버 페이지'},
 'expected_benefits': {'elements': {'BulletPoints': ['고객 만족도 20% 향상', '브랜드 이미지 강화', '법적 리스크 감소'],
                                    'KPIProjection': [['KPI', '현재', '목표'], ['고객 만족도', '70%', '90%'], ['브랜드 인지도', '중간', '높음']],
                                    'Title': '기대 효과'},
                       'needs_research': ['웹 접근성 개선이 고객 만족도에 미치는 영향', '브랜드 이미지 강화 사례'],
                       'research_results': {'웹 접근성 개선이 고객 만족도에 미치는 영향': {'content': '... 접근성을 높일 뿐만 아니라 전반적인 사용자 경험을 개선합니다. 이는 고객 만족도와 고객 유지율을 높이는 데 도움이 될 수 있습니다. 긍정적인 ...\n'
                                                                                    '모바일 개선 접근성 는 모든 능력이 있는 사람들이 디지털 플랫폼을 쉽게 탐색할 수 있도록 함으로써 사용자 경험을 향상시키는 데 중요한 역할을 합니다.\n'
                                                                                    '전환율 증대: 사용성이 개선된 디자인은 고객이 더 쉽게 구매 결정을 내릴 수 있도록 도와줍니다. 결과적으로 전환율이 높아져 매출 증가로 이어질 수 있습니다. 이처럼, '
                                                                                    '...',
                                                                         'urls': ['https://www.ranktracker.com/ko/blog/web-accessibility-why-its-crucial-for-your-digital-marketing-strategy/',
                                                                                  'https://blog.lebara.co.uk/ko/%EC%A0%91%EA%B7%BC%EC%84%B1-%EA%B8%B0%EB%8A%A5%EC%9D%B4-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B2%BD%ED%97%98%EC%97%90-%EB%AF%B8%EC%B9%98%EB%8A%94-%EC%98%81%ED%96%A5%EC%97%90-%EB%8C%80%ED%95%B4-%EC%9E%90/',
                                                                                  'https://epart.com/%EC%82%AC%EC%9A%A9%EC%84%B1-%ED%96%A5%EC%83%81%EC%9C%BC%EB%A1%9C-%EA%B3%A0%EA%B0%9D-%EA%B2%BD%ED%97%98%EC%9D%84-%EC%B5%9C%EC%A0%81%ED%99%94%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95-%EC%A0%91%EA%B7%BC/']}},
                       'slide_description': '웹 접근성 개선을 통해 메리츠종금증권이 얻을 수 있는 구체적인 기대 효과를 제시합니다. 이는 고객 만족도와 브랜드 이미지 향상에 크게 기여할 것입니다.',
                       'slide_title': '기대 효과'},
 'growth_trend_analysis': {'elements': {'GraphLeft': {'data_points': [{'satisfaction': 70, 'year': '2013'}, {'satisfaction': 85, 'year': '2014'}],
                                                      'description': '웹 접근성 개선 후 고객 만족도 변화',
                                                      'graph_type': 'Bar Chart'},
                                        'TextRight': '웹 접근성 개선은 고객 만족도를 15% 이상 향상시킬 수 있으며, 이는 고객 유지율과 신규 고객 유입에 긍정적인 영향을 미칩니다.',
                                        'Title': '성장 추이 분석'},
                           'needs_research': ['웹 접근성 개선 후 고객 만족도 변화 사례', '웹 접근성 개선이 브랜드 이미지에 미치는 영향'],
                           'research_results': {'웹 접근성 개선 후 고객 만족도 변화 사례': {'content': '반복적 테스트 수행: 웹사이트의 접근성이 지속적으로 유지될 수 있도록 정기적으로 테스트합니다. 실제 사용자 경험을 바탕으로 피드백을 받고, 이를 통해 지속적인 '
                                                                                       '개선 ...\n'
                                                                                       '기업 A는 이러한 UX 개선 전략을 통해 방문자 만족도가 30% 증가하고, 전환율이 25% 향상되는 성과를 달성하게 되었습니다. 4.2 사례 연구: 기업 B의 '
                                                                                       '모바일 최적화 전략.\n'
                                                                                       '고객 충성도 증가: 모든 사용자를 배려하는 기업의 모습은 고객 충성도 향상으로 이어질 수 있어요. 언론 보도 기회: 접근성 개선 노력은 긍정적인 언론 ...',
                                                                            'urls': ['https://epart.com/%EC%9B%B9-%EC%A0%91%EA%B7%BC%EC%84%B1-%EA%B0%9C%EC%84%A0%EC%9C%BC%EB%A1%9C-%EB%8D%94-%EB%A7%8E%EC%9D%80-%EC%82%AC%EC%9A%A9%EC%9E%90%EC%97%90%EA%B2%8C-%EB%8B%A4%EA%B0%80%EA%B0%80%EA%B8%B0-%EC%9E%A5/',
                                                                                     'https://epart.com/%EC%9B%B9%EC%82%AC%EC%9D%B4%ED%8A%B8-%EA%B0%9C%EC%84%A0%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B2%BD%ED%97%98ux-%EC%A4%91%EC%8B%AC-%EC%84%A4%EA%B3%84%EC%9D%98-%EC%A4%91%EC%9A%94/',
                                                                                     'https://www.jaenung.net/tree/9176']}},
                           'slide_description': '웹 접근성 개선이 가져올 수 있는 성장 기회를 분석합니다. 이는 고객 만족도와 브랜드 충성도를 높이는 중요한 요소로 작용할 것입니다.',
                           'slide_title': '성장 추이 분석'},
 'implementation_plan': {'elements': {'Phases': ['1단계: 현재 상태 분석 (2014.03 - 2014.04)',
                                                 '2단계: 개선 전략 수립 (2014.05 - 2014.06)',
                                                 '3단계: 솔루션 개발 및 적용 (2014.07 - 2014.09)',
                                                 '4단계: 검증 및 인증 획득 (2014.10 - 2014.11)'],
                                      'TextBottom': '각 단계별로 명확한 목표와 일정이 설정되어 있으며, 이를 통해 프로젝트의 성공적인 완료를 보장합니다.',
                                      'TimelineMain': '프로젝트 실행 타임라인',
                                      'Title': '실행 계획'},
                         'needs_research': ['웹 접근성 개선 프로젝트의 단계별 성공 사례', '효율적인 프로젝트 관리 방법론'],
                         'research_results': {'웹 접근성 개선 프로젝트의 단계별 성공 사례': {'content': "웹 접근성의 4대 원칙인 '인지 가능성', '운용 가능성', '이해 가능성', '견고성'을 기반으로, 실제 사례와 효과적인 개선 방법을 탐구하며, 웹 사이트를 "
                                                                                      '...\n'
                                                                                      '웹사용성 개선은 웹표준 준수, 웹접근성 개선, 도구을 활용한 웹사용성 개. 선, 사용성 평가를 통한 사용성 개선의 순서로 점진적으로 이루어져야 함. i. Page '
                                                                                      '6. 6. 웹 ...\n'
                                                                                      '실태. 조사 결과에 따르면, 2005년 70점대 수준이었던 정. 부기관의 웹 접근성은 2010년 이후 90점대를 상. 회하는 수준으로 향상되었다. 또한, '
                                                                                      '장애인차별금. 지법에 의해 ...',
                                                                           'urls': ['https://honey-dev.com/%EC%9B%B9-%EA%B0%9C%EB%B0%9C%EC%97%90%EC%84%9C%EC%9D%98-%EC%A0%91%EA%B7%BC%EC%84%B1-%EA%B0%9C%EC%84%A0-%EB%B0%A9%EB%B2%95/',
                                                                                    'https://repository.kisti.re.kr/bitstream/10580/12908/1/%EC%A0%9C28%ED%98%B8%2C%20%EC%9B%B9%EC%82%AC%EC%9D%B4%ED%8A%B8%20%EC%82%AC%EC%9A%A9%EC%84%B1%20%EA%B0%9C%EC%84%A0%EC%9D%84%20%EC%9C%84%ED%95%9C%20%EB%8B%A8%EA%B3%84%EB%B3%84%20%EC%A0%84%EB%9E%B5.pdf',
                                                                                    'https://tta.or.kr/data/androReport/ttaJnal/160-7-2.pdf']}},
                         'slide_description': '웹 접근성 개선 프로젝트의 성공적인 실행을 위한 구체적인 계획을 제시합니다. 이는 프로젝트의 효율성과 효과성을 높이는 데 기여할 것입니다.',
                         'slide_title': '실행 계획'},
 'industry_drivers_challenges': {'elements': {'DetailComments': ['법적 요구사항은 지속적으로 강화되고 있으며, 이는 기업들에게 웹 접근성 개선의 필요성을 더욱 강조합니다.', '기술적 구현의 복잡성은 프로젝트의 성공적인 완료를 위한 주요 도전 과제입니다.'],
                                              'Title': '시장 동인 및 도전 과제',
                                              'TwoColumnList': {'Challenges': ['기술적 구현의 복잡성', '기존 시스템과의 통합 문제'], 'Drivers': ['법적 준수 요구 증가', '고객 경험 향상 필요성']}},
                                 'needs_research': ['웹 접근성 관련 법적 요구사항 변화', '기술적 구현의 복잡성을 해결한 사례'],
                                 'research_results': {'웹 접근성 관련 법적 요구사항 변화': {'content': '최근 몇 년 간 웹 접근성을 위한 법적 요구사항이 계속 변화하고 있습니다. 이러한 변화는 웹사이트 운영자와 개발자에게 중요한 의미를 가지며, 모든 사용자가 '
                                                                                         '정보를 ...\n'
                                                                                         "2008년 4월 11일부터 '장애인차별금지 및 권리구제 등에 관한 법률'이 시행되어 사이트의 웹 접근성 준수가 의무화 되었습니다.\n"
                                                                                         '... 접근성 지침 및 규정을 개발했지만 모든 웹 사이트 또는 응용 프로그램이 모든 사람에게 동등하게 작동하도록 하는 보편적인 법적 요구 사항은 없습니다.',
                                                                              'urls': ['https://epart.com/%EC%9B%B9-%EC%A0%91%EA%B7%BC%EC%84%B1%EC%9D%84-%EC%B2%A0%EC%A0%80%ED%9E%88-%EC%A4%80%EC%88%98%ED%95%98%EB%8A%94-%EC%A0%84%EB%9E%B5%EA%B3%BC-%EC%B5%9C%EC%8B%A0-%EB%B2%95%EC%A0%81-%EC%9A%94%EA%B5%AC/',
                                                                                       'https://blog.naver.com/mogni/220711158048?viewType=pc',
                                                                                       'https://jkwondesign.tistory.com/29']}},
                                 'slide_description': '웹 접근성 시장의 주요 동인과 도전 과제를 분석하여 메리츠종금증권이 직면할 수 있는 기회와 위협을 파악합니다.',
                                 'slide_title': '시장 동인 및 도전 과제'},
 'investment_budget_estimation': {'elements': {'BudgetTable': [['항목', '예산(백만 원)'], ['솔루션 개발', '500'], ['인증 획득', '200'], ['교육 및 기술 지원', '100']],
                                               'CostBreakdown': ['솔루션 개발: 500백만 원', '인증 획득: 200백만 원', '교육 및 기술 지원: 100백만 원'],
                                               'Title': '예산 및 비용 추정'},
                                  'needs_research': ['웹 접근성 개선 프로젝트의 비용 절감 사례', '효율적인 예산 관리 방법'],
                                  'research_results': {'웹 접근성 개선 프로젝트의 비용 절감 사례': {'content': '잠재적 시장 점유율 증가, 검색 엔진 최적화(SEO), 사용성 향상으로 인한 트래픽 증가가 가져오는 영업 이익 및 비용 절감 · 소송의 위험, 높은 '
                                                                                              '손해 배상 ...\n'
                                                                                              '따라서 공개SW 기반으로 홈페이지를 전환해 상용SW 업체의 종속적인 유지보수에서 벗어나 비용을 최적화할 수 있었고, 장애나 보안 이슈에 적절히 대응할 '
                                                                                              '...\n'
                                                                                              '이를 위해 접근성 테스트는 중요한 역할을 합니다. 이 블로그 포스트에서는 AI 도구를 활용하여 웹사이트의 접근성을 자동화하고 향상시키는 방법에 대해 '
                                                                                              '알아보겠습니다.',
                                                                                   'urls': ['https://nuli.navercorp.com/community/article/1111811',
                                                                                            'https://www.oss.kr/oss_case/show/19179cdf-4f17-4b35-bf65-94ce48ea91d1',
                                                                                            'https://epart.com/%EC%A0%91%EA%B7%BC%EC%84%B1-%ED%85%8C%EC%8A%A4%ED%8A%B8%EB%A5%BC-%EC%9E%90%EB%8F%99%ED%99%94%ED%95%98%EB%8A%94-ai-%EB%8F%84%EA%B5%AC-%ED%99%9C%EC%9A%A9%EB%B2%95%EC%9C%BC%EB%A1%9C-%EC%9B%B9%EC%82%AC/']}},
                                  'slide_description': '웹 접근성 개선 프로젝트에 필요한 예산과 비용을 추정하여, 프로젝트의 경제성을 평가합니다.',
                                  'slide_title': '예산 및 비용 추정'},
 'market_analysis_market_overview': {'elements': {'GraphLeft': {'data_source': 'Global Web Accessibility Market Report 2023',
                                                                'description': '웹 접근성 시장 성장 추이',
                                                                'graph_type': 'Line Chart'},
                                                  'TextRight': '웹 접근성 시장은 연평균 15% 성장 중이며, 이는 기업들이 법적 준수와 고객 경험 개선을 위해 적극적으로 투자하고 있음을 보여줍니다.',
                                                  'Title': '시장 개요'},
                                     'needs_research': ['웹 접근성 시장의 주요 트렌드 및 성장률', '주요 경쟁사의 웹 접근성 투자 사례'],
                                     'research_results': {'주요 경쟁사의 웹 접근성 투자 사례': {'content': '글로벌 웹 사이트 접근성 소프트웨어 시장 규모는 2024 년에 55 억 달러였으며 2033 년까지 11.3%의 CAGR로 203 억 달러로 증가 할 '
                                                                                             '것으로 예상 ...\n'
                                                                                             "다양한 법률 및 규제 그리고 잠재력이 큰 시장이라는 점으로 인해 2022년에는 '디지털 접근성'이 CIO들의 최우선 순위로 부상할 전망이다.\n"
                                                                                             '이 가이드에서는 경쟁력 분석이 무엇이며 시장 상황에서 회사의 입지를 찾는 것이 왜 비즈니스 전략을 개선할 수 있는지를 설명합니다.',
                                                                                  'urls': ['https://www.businessresearchinsights.com/ko/market-reports/website-accessibility-software-market-101575',
                                                                                           'https://www.cio.com/article/3515759/5%EC%B2%9C%EC%96%B5-%EB%8B%AC%EB%9F%AC-%EC%8B%9C%EC%9E%A5%EC%9D%84-%EB%86%93%EC%B9%A0-%ED%85%90%EA%B0%80-%EC%9B%B9-%EC%A0%91%EA%B7%BC%EC%84%B1%EC%97%90-%EC%A3%BC%EB%AA%A9%ED%95%A0.html',
                                                                                           'https://asana.com/ko/resources/competitive-analysis-example']}},
                                     'slide_description': '웹 접근성 시장의 현황과 트렌드를 분석하여 메리츠종금증권의 전략적 방향성을 제시합니다. 웹 접근성은 고객 경험을 향상시키고 법적 리스크를 줄이는 핵심 요소로 자리잡고 있습니다.',
                                     'slide_title': '시장 개요'},
 'project_understanding': {'elements': {'BackgroundIssues': ['기존 웹사이트의 접근성 지침 미준수', '법/규정 준수를 통한 분쟁 요인 제거 필요'],
                                        'KeyObjectives': ['장애인차별금지법 준수', '고객 사용 편의성 강화', '브랜드 이미지 효과 증대'],
                                        'MiddleText': '웹 접근성 개선의 필요성과 목표',
                                        'Title': '프로젝트 이해'},
                           'needs_research': ['장애인차별금지법 관련 최신 법규 및 사례', '웹 접근성 개선의 경제적/사회적 효과에 대한 연구'],
                           'research_results': {'장애인차별금지법 관련 최신 법규 및 사례': {'content': '제1조(목적) 이 법은 모든 생활영역에서 장애를 이유로 한 차별을 금지하고 장애를 이유로 차별받은 사람의 권익을 효과적으로 구제함으로써 장애인의 완전한 사회참여 '
                                                                                      '...\n'
                                                                                      '이번 토론회는 장애인차별금지법 시행 17주년을 맞아 장애인차별금지법 제정 과정 및 당시의 권리구제 절차와 현재의 장애인차별에 대한 권리구제 기관의 ...',
                                                                           'urls': ['https://www.law.go.kr/lsInfoP.do?lsiSeq=129347',
                                                                                    'https://www.snu.ac.kr/snunow/press?md=v&bbsidx=154585']}},
                           'slide_description': '이 슬라이드는 메리츠종금증권의 웹접근성 개선 프로젝트의 배경과 목표를 명확히 이해시키는 데 중점을 둡니다. 웹 접근성은 단순한 법적 요구를 넘어 고객 경험과 브랜드 이미지를 향상시키는 중요한 요소입니다. 장애인차별금지법 시행에 따라 '
                                                '웹사이트 접근성 준수가 필수적이며, 이를 통해 분쟁을 예방하고 고객 편의성을 높일 수 있습니다.',
                           'slide_title': '프로젝트 이해'},
 'qna': {'elements': {'AnticipatedQuestions': ['웹 접근성 개선의 구체적인 단계는?', '프로젝트 완료 후 유지보수 계획은?'], 'EmptySpace': 'Q&A를 위한 공간', 'Title': '질의응답 세션'},
         'needs_research': ['고객이 자주 묻는 질문과 답변', '웹 접근성 개선 프로젝트의 일반적인 FAQ'],
         'slide_description': '프로젝트와 관련된 질의응답을 통해 고객의 궁금증을 해소하고, 제안의 신뢰성을 높입니다.',
         'slide_title': '질의응답 세션'},
 'risk_management_plan': {'elements': {'MitigationStrategy': ['전문가 컨설팅을 통해 기술적 복잡성을 해결', '법적 요구사항 변화를 지속적으로 모니터링하여 신속히 대응'],
                                       'RiskTable': [['리스크', '영향', '대응 전략'], ['기술적 구현의 복잡성', '높음', '전문가 컨설팅 활용'], ['법적 요구사항 변화', '중간', '지속적 모니터링 및 대응']],
                                       'Title': '리스크 관리 계획'},
                          'needs_research': ['웹 접근성 프로젝트의 주요 리스크 사례', '리스크 관리의 성공적인 사례'],
                          'research_results': {'리스크 관리의 성공적인 사례': '사례: 식품 제조업체의 원재료 공급 리스크. 한 식품 제조업체는 주요 원재료인 특정 곡물의 공급이 기후 변화로 인해 불안정해지는 리스크를 식별 ...\n'
                                                                  '선제적 리스크 관리 시스템 구축의 중요성을 강조하고, 다양한 리스크 관리 기법을 소개합니다. 성공적인 리스크 관리 사례를 통해 실질적인 전략을 제시 ...\n'
                                                                  '금융회사의 수익에 부정적인 영향을 미칠 뿐만 아니라 리스크 머니의. 감소로 인하여 오히려 신용수축이 가속화될 우려도 있다. 2) 기존 법제의 수정. 미국의 경우 임원 ...'},
                          'slide_description': '프로젝트 진행 중 발생할 수 있는 리스크를 사전에 식별하고, 이를 효과적으로 관리하기 위한 계획을 제시합니다.',
                          'slide_title': '리스크 관리 계획'},
 'solution_overview': {'elements': {'MiddleText': '웹 접근성 개선을 위한 통합 솔루션',
                                    'SimpleDiagram': {'description': '솔루션 구현 단계', 'steps': ['현재 상태 분석', '개선 전략 수립', '솔루션 개발 및 적용', '검증 및 인증 획득']},
                                    'Title': '솔루션 개요'},
                       'needs_research': ['웹 접근성 개선을 위한 최신 솔루션 및 기술', '성공적인 웹 접근성 프로젝트 사례'],
                       'research_results': {'성공적인 웹 접근성 프로젝트 사례': {'content': '접근성을 고려해 모든 사용자의 경험을 개선한 실생활의 예시를 들어보자면, 현재 많은 지하철역에 설치되어있는 엘리베이터가 있습니다. 2000년대부터 여러 지하철 역사에 ...\n'
                                                                              '웹 접근성이란 신체 / 환경 / 문화적 차이와 장애 유무에 관계없이 누구나 웹 사이트에 접근할 수 있도록 기술적으로 보장하는 것입니다. 웹 표준에 ...\n'
                                                                              '1. 적절한 대체 텍스트 제공 · 2. 자막 제공 · 3. 색에 무관한 콘텐츠 인식 · 4. 명확한 지시 사항 제공 · 5. 텍스트 콘텐츠의 명도 대비 · 6. 자동 재생 금지.',
                                                                   'urls': ['https://maxkim-j.github.io/posts/web-accessiblity-virtuous-cycle/',
                                                                            'https://m.blog.naver.com/PostView.naver?blogId=musigncorp&logNo=222564165757&navType=by',
                                                                            'https://velog.io/@yesslkim94/211115-211114-%EC%9B%B9%EC%A0%91%EA%B7%BC%EC%84%B1-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-4-%ED%81%AC%EB%A1%9C%EC%8A%A4%EB%B8%8C%EB%9D%BC%EC%9A%B0%EC%A7%95-%EC%9B%B9%EC%A0%91%EA%B7%BC%EC%84%B1']},
                                            '웹 접근성 개선을 위한 최신 솔루션 및 기술': {'content': '웹 접근성이란 물리적, 환경적 장애와 상관 없이 누구나 쉽게 웹 사이트에 접근하여 사용할 수 있는 정도를 의미합니다. 본 포스트를 통해 웹 접근성 ...\n'
                                                                                    'All in One Accessibility®. WCAG 2.0, 2.1, 2.2 및 ADA 개선을 위한 AI 기반 웹사이트 접근성 솔루션! 웹사이트 접근성을 빠르게 '
                                                                                    '개선하기 위한 접근성 솔루션입니다.\n'
                                                                                    '따라서 웹과 모바일 환경에서 접근성을 고려하는 것은 중요합니다. 이는 글자 크기를 조정하거나, 색상 대비를 개선하고, 음성 인식 기능을 활용하는 등의 ...',
                                                                         'urls': ['https://seo.tbwakorea.com/blog/web-accessibility/',
                                                                                  'https://www.skynettechnologies.com/korean-website-accessibility-solution',
                                                                                  'https://m.emotion.co.kr/magazine/51']}},
                       'slide_description': '메리츠종금증권의 웹 접근성 개선을 위한 솔루션의 전반적인 개요를 제시합니다. 이는 고객 경험을 향상시키고 법적 요구사항을 충족시키는 데 중점을 둡니다.',
                       'slide_title': '솔루션 개요'},
 'strategic_recommendations': {'elements': {'BulletPoints': ['웹 접근성 인증 획득을 위한 단계별 접근', '모바일 및 다양한 브라우저 호환성 강화', '고객 피드백을 반영한 지속적 개선'],
                                            'PriorityMap': [['전략', '우선순위', '실행 시기'], ['웹 접근성 인증', '높음', '즉시'], ['모바일 호환성', '중간', '중기'], ['고객 피드백 반영', '높음', '지속']],
                                            'Title': '전략 제안'},
                               'needs_research': ['웹 접근성 인증 획득을 위한 구체적 절차', '모바일 호환성 강화를 위한 기술적 방안'],
                               'slide_description': '메리츠종금증권의 웹 접근성 개선을 위한 구체적인 전략을 제안합니다. 이는 법적 준수와 고객 경험 향상을 동시에 달성할 수 있는 방향성을 제시합니다.',
                               'slide_title': '전략 제안'},
 'swot_analysis': {'elements': {'NarrativeSummary': '메리츠종금증권은 강력한 브랜드 이미지를 바탕으로 웹 접근성을 개선함으로써 고객 신뢰도를 높일 수 있습니다. 그러나 기술적 자원의 부족은 해결해야 할 과제입니다.',
                                'SWOTTable': {'Opportunities': ['법적 준수로 인한 신뢰도 향상', '모바일 사용 증가'],
                                              'Strengths': ['고객 중심의 서비스 제공', '강력한 브랜드 이미지'],
                                              'Threats': ['경쟁사의 빠른 기술 도입', '법적 요구사항의 지속적 강화'],
                                              'Weaknesses': ['현재 웹 접근성 미흡', '기술적 자원 부족']},
                                'Title': 'SWOT 분석'},
                   'needs_research': ['웹 접근성 개선의 내부 강점 및 약점 분석', '외부 기회와 위협 요소 분석'],
                   'research_results': {'외부 기회와 위협 요소 분석': {'content': 'SWOT 분석은 내부 환경과 외부 환경 요소를 기반으로 현황을 분석하는 방법입니다. 자사가 놓치고 있는 잠재적 가능성과 강점을 발견하는 방법입니다.\n'
                                                                       'SWOT 분석은 비즈니스나 특정 프로젝트의 강점, 단점, 기회, 위협을 식별하기 위해 사용하는 기법입니다. SWOT 분석은 소규모 비즈니스와 비영리 조직부터 ...\n'
                                                                       'SWOT는 Strength(강점), Weakness(약점), Opportunity(기회), Threat(위협)의 4가지 요소로 구성되어 있습니다. SWOT 분석에서는 위 4가지 요소 중 '
                                                                       '세로축 ...',
                                                            'urls': ['https://www.salesforce.com/kr/hub/marketing/swot/',
                                                                     'https://asana.com/ko/resources/swot-analysis',
                                                                     'https://blog.naver.com/brandkim/223576090829?viewType=pc']},
                                        '웹 접근성 개선의 내부 강점 및 약점 분석': {'content': '개발이 더 쉬워집니다 — 앞서 언급했듯이 내장된 몇몇 기능을 활용할 수 있으며, 이해하기에도 확실히 더 쉽습니다. · 모바일 환경에서 더 좋습니다 — ...\n'
                                                                               "웹 접근성: 웹 접근성은 의심의 여지 없이 프론트엔드 테스팅에서 필수적으로 고려해야 할 사항이다. 간단하게 말해서 '누구나' 웹 페이지를 접근하고 ...\n"
                                                                               '웹 접근성을 향상하면 사용자 친화적인 인터페이스가 구축되기 때문에 특정 대상이 아닌, 모든 사용자에게 긍정적인 영향을 끼칠 수 있습니다. 즉, 사실상 ...',
                                                                    'urls': ['https://developer.mozilla.org/ko/docs/Learn_web_development/Core/Accessibility/HTML',
                                                                             'https://velog.io/@kandy1002/%EC%9B%B9-%EC%A0%91%EA%B7%BC%EC%84%B1%EA%B3%BC-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B2%BD%ED%97%98%EC%9D%84-%EA%B3%A0%EB%A0%A4%ED%95%9C-%ED%94%84%EB%A1%A0%ED%8A%B8%EC%97%94%EB%93%9C-%ED%85%8C%EC%8A%A4%ED%8A%B8-%EC%BD%94%EB%93%9C-%EC%9E%91%EC%84%B1%ED%95%98%EA%B8%B0',
                                                                             'https://seo.tbwakorea.com/blog/web-accessibility/']}},
                   'slide_description': '메리츠종금증권의 웹 접근성 개선 프로젝트에 대한 SWOT 분석을 통해 전략적 방향성을 제시합니다.',
                   'slide_title': 'SWOT 분석'},
 'table_of_contents': {'elements': {'SectionList': ['I. 사업개요', 'II. 제안요청 내용', 'III. 제안안내', 'IV. 참여조건 및 방법', 'V. 첨부'], 'Title': '목차'}, 'slide_title': '목차'},
 'team_introduction': {'elements': {'MemberList': [{'experience': '10년', 'name': '김철수', 'role': '프로젝트 매니저'},
                                                   {'experience': '8년', 'name': '이영희', 'role': '웹 개발자'},
                                                   {'experience': '5년', 'name': '박민수', 'role': '디자이너'}],
                                    'TeamOrgChart': '팀 조직도 이미지',
                                    'Title': '팀 소개'},
                       'needs_research': ['성공적인 프로젝트 팀 구성 사례', '팀 구성원의 역할과 책임'],
                       'research_results': {'성공적인 프로젝트 팀 구성 사례': {'content': '강력하고 응집력 있는 팀을 구축하기 위한 모범 사례 · 팀 인정 - 이정표나 성공을 달성하는 데 기여한 각 팀원의 공로를 인정합니다. · 원격 작업 - 팀원들이 원격으로 ...\n'
                                                                             '팀 구조: 팀을 구성하는 10가지 효과적인 방법 · 1. 계층적 구조 · 2. 기능적 구조 · 3. 매트릭스 구조 · 4. 프로세스 기반 구조 · 5. 원형 구조 · 6. 평면적 '
                                                                             '...\n'
                                                                             '사례 ... 올바른 팀을 선택하는 것은 프로젝트의 성공에 필수적입니다. 당면한 특정 프로젝트에 필요한 기술과 전문 지식을 기반으로 팀을 구성합니다.',
                                                                  'urls': ['https://thecodest.co/ko/blog/%EA%B0%95%EB%A0%A5%ED%95%98%EA%B3%A0-%EC%9D%91%EC%A7%91%EB%A0%A5-%EC%9E%88%EB%8A%94-%ED%8C%80%EC%9D%84-%EA%B5%AC%EC%B6%95%ED%95%98%EA%B8%B0-%EC%9C%84%ED%95%9C-%EB%AA%A8%EB%B2%94-%EC%82%AC%EB%A1%80/',
                                                                           'https://asana.com/ko/resources/team-structure',
                                                                           'https://www.slingshotapp.io/ko/blog/project-management-strategies-for-successful-execution']},
                                            '팀 구성원의 역할과 책임': {'content': '팀의 역할을 정의하면 팀 효율이 높아집니다. 이 가이드에서는 역할과 책임을 정립하는 방법을 소개하고 역할과 책임을 정립하는 것이 팀에 유익한 ...\n'
                                                                         '팀장은 특정 그룹, 팀, 조직 또는 프로젝트를 관리하는 역할을 할 수 있습니다. 그들이 업무를 수행하는 방식은 팀의 생산성과 성과에 막대한 영향을 미칠 수 있습니다 ...\n'
                                                                         '팀 역할(Team role)이란 직무와 직접적인 관련은 없지만 구성원 간의 긍정적인 상호작용을 일으키는 역할을 의미합니다. 벨빈 교수는 팀 역할을 크게 사고 ...',
                                                              'urls': ['https://asana.com/ko/resources/roles-and-responsibilities',
                                                                       'https://brunch.co.kr/@exceo/33',
                                                                       'https://blog.naver.com/donaeun/223493061687?viewType=pc']}},
                       'slide_description': '프로젝트를 성공적으로 이끌어갈 팀의 구성과 역할을 소개합니다. 이는 고객에게 신뢰를 줄 수 있는 중요한 요소입니다.',
                       'slide_title': '팀 소개'},
 'timeline_milestones': {'elements': {'GanttChart': {'description': '프로젝트 타임라인',
                                                     'phases': [{'end': '2014.04', 'phase': '현재 상태 분석', 'start': '2014.03'},
                                                                {'end': '2014.06', 'phase': '개선 전략 수립', 'start': '2014.05'},
                                                                {'end': '2014.09', 'phase': '솔루션 개발 및 적용', 'start': '2014.07'},
                                                                {'end': '2014.11', 'phase': '검증 및 인증 획득', 'start': '2014.10'}]},
                                      'Title': '타임라인 및 주요 마일스톤'},
                         'needs_research': ['효과적인 프로젝트 타임라인 관리 사례', '주요 마일스톤 설정의 중요성 및 사례'],
                         'research_results': {'주요 마일스톤 설정의 중요성 및 사례': {'content': '마일스톤을 설정하면 목표로 한 날짜에 집중하고 필요에 따라 프로젝트 계획을 쉽게 조정할 수 있습니다. 마일스톤을 활용하여 즐거움을 더할 수도 있습 ...\n'
                                                                                  '각 마일스톤은 특정한 결과물이나 성과를 달성하기 위한 중요한 마크로서, 주로 시간, 비용, 성과 등의 측면에서 설정되며, 전체적인 진행상황이나 성공 ...\n'
                                                                                  '스타트업의 성공적인 마일스톤 설정을 위한 SMART 방법론을 소개합니다. 이 방법론은 구체성, 측정가능성, 달성가능성, 관련성, 시간 제약의 5가지 ...',
                                                                       'urls': ['https://asana.com/ko/resources/project-milestones',
                                                                                'https://www.waveon.io/blog/milestone-1',
                                                                                'https://www.waveon.io/blog/milestone-2']},
                                              '효과적인 프로젝트 타임라인 관리 사례': {'content': '또 monday.com 업무 관리 툴을 사용하여 프로젝트 타임라인을 신속하고 효율적으로 구축할 수 있는 방법과 실제 사례에 대해 상세히 소개합니다.\n'
                                                                                  '타임라인은 시작 날짜 및 종료 날짜가 정의된 정확한 일정 수립을 통해 과로를 방지합니다. 작업 종속성은 올바른 작업 순서를 위한 상호 의존성을 강조합니다. 진행률 ...\n'
                                                                                  '1. 프로젝트 브리프를 작성합니다 · 2. 할 일 목록을 작성합니다 · 3. 각 단계의 소요 기간을 예측합니다 · 4. 종속 관계를 계획합니다 · 5. 타임라인을 생성 '
                                                                                  '...',
                                                                       'urls': ['https://monday.com/blog/ko/unclassified/project-timeline/',
                                                                                'https://www.atlassian.com/ko/agile/project-management/gantt-chart-examples',
                                                                                'https://asana.com/ko/resources/create-project-management-timeline-template']}},
                         'slide_description': '프로젝트의 주요 마일스톤을 명확히 제시하여, 각 단계의 진행 상황을 효과적으로 관리할 수 있도록 합니다.',
                         'slide_title': '타임라인 및 주요 마일스톤'},
 'why_us_differentiation': {'elements': {'BulletPoints': ['풍부한 웹 접근성 개선 경험', '맞춤형 솔루션 제공', '고객 중심의 서비스'],
                                         'ComparisonTable': [['항목', 'EY 컨설팅', '경쟁사 A'], ['경험', '풍부', '보통'], ['솔루션', '맞춤형', '일반적'], ['고객 서비스', '고객 중심', '보통']],
                                         'Title': '당사를 선택해야 하는 이유'},
                            'needs_research': ['EY 컨설팅의 웹 접근성 개선 성공 사례', '경쟁사와의 차별화 요소'],
                            'slide_description': 'EY 컨설팅이 메리츠종금증권의 웹 접근성 개선 프로젝트에 적합한 파트너임을 증명하는 차별화된 강점을 제시합니다.',
                            'slide_title': '당사를 선택해야 하는 이유'}} 

In [93]:
import openai
from copy import deepcopy
import ast

def refine_slides_dict(slides_dict, rfp_text, user_inputs):
    refined_slides = deepcopy(slides_dict)

    for slide_id, slide in slides_dict.items():
        try:
            prompt = f"""
            당신은 맥킨지 수준의 컨설팅 제안서 전략 전문가입니다.
            
            당신의 역할은 제안서 슬라이드의 내용을 전략적으로 더 설득력 있고 고도화된 형태로 다듬는 것입니다.
            
            --- 컨텍스트 ---
            클라이언트: {user_inputs['client_name']}
            제안서 제목: {user_inputs['proposal_title']}
            문체 및 톤: {user_inputs['style']}
            작성 방향: {user_inputs['user_direction']}
            강조할 키워드: {", ".join(user_inputs['keywords'])}
            RFP 원문: {rfp_text}
            
            --- 슬라이드 정보 ---
            슬라이드 ID: {slide_id}
            슬라이드 제목: {slide.get("slide_title", "")}
            현재 슬라이드 설명: {slide.get("description", "")}
            슬라이드 요소: {slide.get("elements", {})}
            리서치 결과: {slide.get("research_results", {})}
            
            --- 작업 지시 ---
            다음의 정확한 키를 포함하는 유효한 Python 딕셔너리 하나만 반환하세요:
            - "description": 현재 슬라이드 설명을 기반으로 더욱 전략적이고 설득력 있게 다듬은 설명 (3~5문장)
            - "elements": 기존 요소를 기반으로 논리적 구조와 설득력을 강화한 내용. 특히 리스트, 표, 그래프 등은 Research Results를 기반으로 실제 수치를 반영하여 구체적으로 작성하세요.
            
            ⚠️ [중요] 표나 그래프가 필요한 경우:
            - 표는 리스트로 구성하세요. 예: [["헤더1", "헤더2"], ["값1", "값2"], ...]
            - 그래프는 다음 형식으로 작성하세요. 반드시 적절한 그래프 타입을 데이터 구조에 맞게 지정해야 합니다:
              "GraphLeft": {{
                "description": "그래프 설명",
                "graph_type": "Line Chart" 또는 "Bar Chart" 또는 "Pie Chart" 중 하나,
                "data_source": "출처 예: 통계청, 2024",
                "data_table": [["헤더1", "헤더2"], ["값1", "값2"], ...]
              }}
            
            ⚠️ 반드시 리서치 결과를 바탕으로 수치를 정리하고, 자료 출처를 명시해야 합니다.
            
            ⚠️ 다음 조건을 꼭 지켜주세요:
            - 마크다운, 인용 부호, 코드 블록, 주석 등을 추가하지 마세요
            - 반드시 한글로만 작성하세요
            - 반환 형식은 Python 딕셔너리 하나만 출력하세요
            """



            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt.strip()}],
                temperature=0.3
            )
            result_text = response.choices[0].message.content.strip()
            
            # 안전한 파싱 (JSON이 아닌 파이썬 dict로)
            parsed = ast.literal_eval(result_text)

            refined_slides[slide_id]["slide_description"] = parsed.get("slide_description", "nan")
            refined_slides[slide_id]["elements"] = parsed.get("elements", {})

        except Exception as e:
            print(f"[고도화 실패] {slide_id}: {e}")
            refined_slides[slide_id]["slide_description"] = "nan"
            refined_slides[slide_id]["elements"] = {}

    return refined_slides


In [94]:
refined = refine_slides_dict(
    slides_dict=slides_dict,
    rfp_text=rfp_text,
    user_inputs=user_inputs,
)

In [ ]:
refined

In [101]:
refined

{'cover_page': {'slide_description': 'nan',
  'description': '메리츠종금증권 웹접근성 개선 프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다. 이는 프로젝트의 중요성과 고객사의 브랜드 이미지를 강조하는 첫인상입니다.',
  'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2014.02',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 및 메리츠종금증권 로고'}},
 'table_of_contents': {'slide_description': 'nan',
  'description': '제안서의 전체 흐름을 한눈에 파악할 수 있도록 목차를 제공하여 독자가 쉽게 원하는 정보를 찾을 수 있도록 합니다. 이는 제안서의 체계성과 논리적 구성을 강조합니다.',
  'elements': {'Title': '목차',
   'SectionList': ['I. 사업개요',
    'II. 제안요청 내용',
    'III. 제안안내',
    'IV. 참여조건 및 방법',
    'V. 첨부']}},
 'project_understanding': {'slide_description': 'nan',
  'description': '웹 접근성 개선의 필요성과 목표를 명확히 설명하여 프로젝트의 중요성과 시급성을 강조합니다. 특히 장차법 시행에 따른 법적 준수 필요성과 고객 사용 편의성 강화의 중요성을 부각합니다.',
  'elements': {'Title': '프로젝트 이해',
   'MiddleText': '웹 접근성 준수를 통한 법적 리스크 제거 및 사용자 편의성 강화',
   'KeyObjectives': ['웹 접근성 인증마크 획득', '장애인 접근성 강화', '브랜드 이미지 제고'],
   'BackgroundIssues': ['장차법 시행에 따른

In [100]:
slides = refined
for slide_id, slide_info in slides_dict.items():
   print(slide_id, slide_info)
# slides는 refined 상태라고 가정
ordered_keys = [
    "cover_page",
    "table_of_contents",
    "project_understanding",
    "client_needs_summary",
    "market_analysis_market_overview",
    "growth_trend_analysis",
    "industry_drivers_challenges",
    "competitive_benchmarking",
    "swot_analysis",
    "solution_overview",
    "strategic_recommendations",
    "implementation_plan",
    "timeline_milestones",
    "expected_benefits",
    "investment_budget_estimation",
    "risk_management_plan",
    "team_introduction",
    "why_us_differentiation",
    "qna",
    "closing_summary"
]

# slides를 순서에 맞게 재정렬
slides = refined  # 기존 슬라이드 딕셔너리
sorted_slides = {k: slides[k] for k in ordered_keys if k in slides}

# 결과 확인
for slide_id, slide_info in sorted_slides.items():
    print(slide_id, slide_info)


cover_page {'slide_description': '프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다.', 'description': '메리츠종금증권 웹접근성 개선 프로젝트의 시작을 알리는 커버 페이지로, 프로젝트명과 제출 정보를 포함합니다. 이는 프로젝트의 중요성과 고객사의 브랜드 이미지를 강조하는 첫인상입니다.', 'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트 시스템 제안서', 'Subtitle': 'EY 컨설팅', 'ProjectDate': '2014.02', 'PreparedBy': 'EY 컨설팅', 'Logo': 'EY 및 메리츠종금증권 로고'}}
table_of_contents {'slide_description': '전체 제안서의 구성 흐름을 한눈에 보여주는 목차입니다.', 'description': '제안서의 전체 흐름을 한눈에 파악할 수 있도록 목차를 제공하여 독자가 쉽게 원하는 정보를 찾을 수 있도록 합니다. 이는 제안서의 체계성과 논리적 구성을 강조합니다.', 'elements': {'Title': '목차', 'SectionList': ['I. 사업개요', 'II. 제안요청 내용', 'III. 제안안내', 'IV. 참여조건 및 방법', 'V. 첨부']}}
project_understanding {'slide_description': '프로젝트의 배경 및 목적을 설명하고 고객의 니즈에 대한 이해를 보여줍니다.', 'description': '웹 접근성 개선의 필요성과 목표를 명확히 설명하여 프로젝트의 중요성과 시급성을 강조합니다. 특히 장차법 시행에 따른 법적 준수 필요성과 고객 사용 편의성 강화의 중요성을 부각합니다.', 'elements': {'Title': '프로젝트 이해', 'MiddleText': '웹 접근성 준수를 통한 법적 리스크 제거 및 사용자 편의성 강화', 'KeyObjectives': ['웹 접근성 인증마크 획득', '장애인 접근성 강화', '브랜드 이미지

In [97]:
# 전체 슬라이드 데이터를 모두 사용하여 순서대로 정렬
# 사용자가 제공한 전체 딕셔너리를 변수로 할당
from collections import OrderedDict
import pandas as pd
import ace_tools as tools

# 실제 전체 데이터는 이미 사용자 메시지로 제공되었으며, 여기에 재정의
import json

# 전체 슬라이드 키 순서
ordered_keys = [
    "cover_page",
    "table_of_contents",
    "project_understanding",
    "client_needs_summary",
    "market_analysis_market_overview",
    "growth_trend_analysis",
    "industry_drivers_challenges",
    "competitive_benchmarking",
    "swot_analysis",
    "solution_overview",
    "strategic_recommendations",
    "implementation_plan",
    "timeline_milestones",
    "expected_benefits",
    "investment_budget_estimation",
    "risk_management_plan",
    "team_introduction",
    "why_us_differentiation",
    "qna",
    "closing_summary"
]

# 전체 데이터: 이 셀에 사용자 제공 전체 dict 직접 삽입 (길이 제한으로 별도 수동 삽입 생략)
# 지금은 이미 임시로 저장해둔 전체 데이터를 활용
from IPython import get_ipython

# 사용자 메시지에서 전체 데이터가 input에 포함되어 있다고 가정
ip = get_ipython()
user_ns = ip.user_ns
full_data = user_ns.get('data_full')  # 이 변수로 전체 슬라이드 데이터 주입되었다고 가정

# 데이터가 존재할 경우 정렬 수행
if full_data:
    ordered_slides = OrderedDict()
    for key in ordered_keys:
        if key in full_data:
            ordered_slides[key] = full_data[key]

    df = pd.DataFrame([
        {"slide_id": key, "slide_title": val.get("slide_title", ""), "description": val.get("slide_description", "")}
        for key, val in ordered_slides.items()
    ])
    tools.display_dataframe_to_user(name="전체 슬라이드 정렬 결과", dataframe=df)
else:
    raise ValueError("전체 슬라이드 데이터가 'data_full' 변수에 존재하지 않습니다.")


ModuleNotFoundError: No module named 'ace_tools'

{'cover_page': {'slide_title': '커버 페이지',
  'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트',
   'Subtitle': 'EY 컨설팅의 맞춤형 시스템 제안',
   'Key Highlights': ['타사 비교를 통한 벤치마크 분석',
    '자체적인 접근성 개선 전략',
    '현실적이고 실행 가능한 설계'],
   'Confidentiality Notice': '본 제안서는 메리츠종금증권 웹접근성 개선 프로젝트를 위한 목적으로만 사용되며, 외부 또는 타사에 배포할 수 없습니다.'},
  'slide_description': "This proposal outlines a strategic approach to enhancing web accessibility for Meritz Securities, leveraging EY Consulting's expertise. Our solution is grounded in comprehensive benchmarking and comparative analysis with industry peers, ensuring a robust and sustainable implementation. We aim to deliver a tailored system that aligns with Meritz Securities' operational realities and strategic objectives."},
 'table_of_contents': {'slide_title': '목차',
  'elements': {'I. 사업개요': {'1. 추진배경 및 목표': '프로젝트의 필요성과 목표를 명확히 정의하여 이해 관계자 모두가 공감할 수 있는 방향성을 제시합니다.',
    '2. 사업내용': '프로젝트의 주요 활동과 기대 효과를 구체적으로 설명하여 실현 가능한 결과를 도출합니다.'},
   'II. 제안요청 내용': {'1. 제안 기본사항': '제안의 